# 删除库

In [10]:
subdir = [
    # "aan_1",
    # "address_1",
    # "boat_1",
    # "online_exams",
    # "video_game",
    # "government_shift",
    # "planet_1",
    # "warehouse_1",
    # "book_press",
    # "district_spokesman",
    # "restaurant_bills",
    # "soccer_3",
    # "tv_shows",
    # "institution_sports",
    # "car_road_race",
    # "bakery_1",
    # "bbc_channels",
    # "region_building",
    "cre_doc_workflow",
    # "pilot_1",
    # "country_language",
    # "club_leader",
    # "cre_doc_and_collections",
    # "book_1",
    # "cre_students_information_systems",
    # "customers_and_orders",
    # "advertising_agencies", 
    # "university_rank",
    # "sing_contest",
    # "movie_2",
    # "bike_racing",
    # "e_commerce",
    # "art_1", 
    # "real_estate_rentals",
    # "book_review",
    # "car_racing",
    # "vehicle_driver",
    # "headphone_store",
    # "conference",
    # "vehicle_rent",
]
# subdir = [
#     "aan_1",
# ]

In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# ... DB_CONFIG 应该连接到 'postgres' 数据库 ...

def force_drop_database(db_name, config):
    conn = None
    try:
        conn = psycopg2.connect(**config)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()
        
        # 1. 终止所有连接
        terminate_query = f"""
        SELECT pg_terminate_backend(pid)
        FROM pg_stat_activity
        WHERE datname = '{db_name}' AND pid <> pg_backend_pid();
        """
        print(f"尝试终止所有连接到 {db_name} 的进程...")
        cursor.execute(terminate_query)
        print("连接终止完成。")

        # 2. 执行删除操作
        drop_query = f"DROP DATABASE {db_name};"
        print(f"尝试删除数据库: {db_name}...")
        cursor.execute(drop_query)
        print(f"✅ 数据库 '{db_name}' 成功删除。")

    except psycopg2.Error as e:
        print(f"❌ 删除数据库 '{db_name}' 失败: {e}")
    finally:
        if conn:
            conn.close()
DB_CONFIG = {
        "host": "xxx",
        "user": "postgres",      # 替换为您的用户名
        "password": "xxx",  # 替换为您的密码
        "port": "xxx"               # 默认端口
    }
for i in subdir: 
     force_drop_database(f"spider1_0_{i}", DB_CONFIG)
# 示例调用
# force_drop_database("spider1_0_warehouse_1", ADMIN_DB_CONFIG)

尝试终止所有连接到 spider1_0_cre_doc_workflow 的进程...
连接终止完成。
尝试删除数据库: spider1_0_cre_doc_workflow...
✅ 数据库 'spider1_0_cre_doc_workflow' 成功删除。


# 批量创建数据库

In [ ]:
import psycopg2
import os
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
def create_db(database):
    db_list = []
    for i in database:
        db_list.append(f"spider1_0_{i}")
    # --- 1. 数据库连接配置 ---
    DB_CONFIG = {
        "host": "xxx",
        "database": "postgres",  # 替换为您的数据库名称
        "user": "postgres",      # 替换为您的用户名
        "password": "xxx",  # 替换为您的密码
        "port": "xxx"               # 默认端口
    }

    # --- 2. SQL 文件路径 ---
    SQL_FILE_PATH = f"_{database}-schema.sql"  # 替换为您的 SQL 文件路径

    def create_databases(db_list, config):
        """
        连接到 PostgreSQL 管理数据库并批量创建新的数据库。
        """
        conn = None
        try:
            # 1. 建立管理连接
            print(f"尝试连接到 PostgreSQL 服务器 ({config['database']})...")
            conn = psycopg2.connect(**config)
            
            # 2. 设置隔离级别为 AUTOCOMMIT
            # CREATE DATABASE 命令不能在事务块中执行，必须使用 AUTOCOMMIT 模式。
            conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
            cursor = conn.cursor()
            print("连接成功，启用 AUTOCOMMIT 模式。")
            
            # 3. 逐个创建数据库
            for db_name in db_list:
                # 确保数据库名称是安全的，防止 SQL 注入
                # 这里简单使用字符串格式化，但 CREATE DATABASE 不能使用参数化查询
                create_query = f"CREATE DATABASE {db_name};"
                
                try:
                    print(f"正在创建数据库: {db_name}...")
                    cursor.execute(create_query)
                    print(f"✅ 数据库 '{db_name}' 创建成功。")
                    
                except psycopg2.errors.DuplicateDatabase:
                    print(f"⚠️ 数据库 '{db_name}' 已存在，跳过。")
                
                except psycopg2.Error as e:
                    print(f"❌ 创建数据库 '{db_name}' 失败: {e}")
                    # 即使失败也继续尝试下一个数据库
                    
        except psycopg2.Error as e:
            print(f"\n❌ 致命错误：无法连接到 PostgreSQL 服务器。请检查配置和凭证。")
            print(e)
            
        finally:
            # 4. 关闭连接
            if conn:
                cursor.close()
                conn.close()
                print("\n数据库连接已关闭。")


    create_databases(db_list, DB_CONFIG)

In [13]:
create_db(subdir)

尝试连接到 PostgreSQL 服务器 (postgres)...
连接成功，启用 AUTOCOMMIT 模式。
正在创建数据库: spider1_0_cre_doc_workflow...
✅ 数据库 'spider1_0_cre_doc_workflow' 创建成功。

数据库连接已关闭。


# 提交脚本

In [ ]:
import psycopg2
import os

def commit(database):
    database = f"spider1_0_{database}"
    # --- 1. 数据库连接配置 ---
    DB_CONFIG = {
        "host": "xxx",
        "database": database,  # 替换为您的数据库名称
        "user": "postgres",      # 替换为您的用户名
        "password": "xxx",  # 替换为您的密码
        "port": "xxx"               # 默认端口
    }

    # --- 2. SQL 文件路径 ---
    SQL_FILE_PATH = f"{database}-schema.sql"  # 替换为您的 SQL 文件路径

    def execute_sql_file():
        conn = None
        try:
            # 连接到 PostgreSQL 数据库
            print(f"尝试连接到数据库: {DB_CONFIG['database']}...")
            conn = psycopg2.connect(**DB_CONFIG)
            conn.autocommit = False  # 禁用自动提交，以实现事务控制
            cursor = conn.cursor()
            print("连接成功。")

            # 读取 SQL 文件内容
            print(f"读取 SQL 文件: {SQL_FILE_PATH}...")
            with open(SQL_FILE_PATH, 'r', encoding='utf8') as f:
                # PostgreSQL 的 psycop2 驱动不能像 MySQL 一样直接处理多条语句
                # 我们需要将文件内容拆分成单条命令
                sql_commands = f.read()

            # 使用 split() 方法按分号 ; 拆分命令，并过滤掉空行
            # 注意：这假设您的 SQL 文件中的每条命令都以 ; 结束
            commands = [cmd.strip() for cmd in sql_commands.split(';') if cmd.strip()]

            print(f"找到 {len(commands)} 条 SQL 命令，开始执行...")

            # 逐条执行 SQL 命令
            for command in commands:
                # 过滤掉注释行
                if command.upper().startswith('--') or command.upper().startswith('/*'):
                    continue

                try:
                    # 打印正在执行的命令（仅打印前100字符）
                    print(f"执行: {command[:100]}...")
                    cursor.execute(command)
                    
                except psycopg2.Error as e:
                    # 如果某条命令失败，打印错误并继续或中止（取决于需求）
                    print(f"❌ 命令执行失败: {e.diag.message_primary}")
                    # 默认行为：遇到错误则抛出异常，触发 finally 中的回滚

            # 提交事务
            conn.commit()
            print("\n✅ 所有 SQL 命令在一个事务中成功执行并提交。")

        except psycopg2.OperationalError as e:
            print(f"\n❌ 连接或操作失败: {e}")
            print("请检查数据库配置、服务状态和网络连接。")
            
        except FileNotFoundError:
            print(f"\n❌ 错误: 找不到 SQL 文件 {SQL_FILE_PATH}，请检查路径。")

        except Exception as e:
            print(f"\n❌ 发生意外错误: {e}")
            if conn:
                conn.rollback()  # 遇到任何错误都回滚事务
                print("事务已回滚。")

        finally:
            # 关闭数据库连接
            if conn:
                cursor.close()
                conn.close()
                print("数据库连接已关闭。")


    execute_sql_file()

In [15]:
for i in subdir:
    commit(i)

尝试连接到数据库: spider1_0_cre_doc_workflow...
连接成功。
读取 SQL 文件: spider1_0_cre_doc_workflow-schema.sql...
找到 9 条 SQL 命令，开始执行...
执行: CREATE TABLE "authors" (
  "author_name" varchar(255)  NOT NULL,
  "other_details" varchar(255)  NOT...
执行: CREATE TABLE "business_processes" (
  "process_id" INT GENERATED BY DEFAULT AS IDENTITY,
  "next_pro...
执行: CREATE TABLE "process_outcomes" (
  "process_outcome_code" char(15)  NOT NULL,
  "process_outcome_de...
执行: CREATE TABLE "process_status" (
  "process_status_code" char(15)  NOT NULL,
  "process_status_descri...
执行: CREATE TABLE "ref_staff_roles" (
  "staff_role_code" char(15)  NOT NULL,
  "staff_role_description" ...
执行: CREATE TABLE "staff" (
  "staff_id" INT GENERATED BY DEFAULT AS IDENTITY,
  "staff_details" varchar(...
执行: CREATE TABLE "documents" (
  "document_id" INT GENERATED BY DEFAULT AS IDENTITY,
  "author_name" var...
执行: CREATE TABLE "documents_processes" (
  "document_id" int NOT NULL,
  "process_id" int NOT NULL,
  "p...
执行: CREATE TABLE

# 插入数据

In [ ]:
import psycopg2
import os

def commit2(database):
    database = f"spider1_0_{database}"
    # --- 1. 数据库连接配置 ---
    DB_CONFIG = {
        "host": "xxx",
        "database": database,  # 替换为您的数据库名称
        "user": "postgres",      # 替换为您的用户名
        "password": "xxx",  # 替换为您的密码
        "port": "xxx"               # 默认端口
    }

    # --- 2. SQL 文件路径 ---
    SQL_FILE_PATH = f"__{database}-data.sql"  # 替换为您的 SQL 文件路径

    def execute_sql_file():
        conn = None
        try:
            # 连接到 PostgreSQL 数据库
            print(f"尝试连接到数据库: {DB_CONFIG['database']}...")
            conn = psycopg2.connect(**DB_CONFIG)
            conn.autocommit = False  # 禁用自动提交，以实现事务控制
            cursor = conn.cursor()
            print("连接成功。")

            # 读取 SQL 文件内容
            print(f"读取 SQL 文件: {SQL_FILE_PATH}...")
            with open(SQL_FILE_PATH, 'r', encoding='utf8') as f:
                # PostgreSQL 的 psycop2 驱动不能像 MySQL 一样直接处理多条语句
                # 我们需要将文件内容拆分成单条命令
                sql_commands = f.read()

            # 使用 split() 方法按分号 ; 拆分命令，并过滤掉空行
            # 注意：这假设您的 SQL 文件中的每条命令都以 ; 结束
            commands = [cmd.strip() for cmd in sql_commands.split(';') if cmd.strip()]
            print(commands)
            print(f"找到 {len(commands)} 条 SQL 命令，开始执行...")

            # 逐条执行 SQL 命令
            for command in commands:
                # 过滤掉注释行
                if command.upper().startswith('--') or command.upper().startswith('/*'):
                    continue

                try:
                    # 打印正在执行的命令（仅打印前100字符）
                    print(f"执行: {command[:100]}...")
                    cursor.execute(command)
                    
                except psycopg2.Error as e:
                    # 如果某条命令失败，打印错误并继续或中止（取决于需求）
                    print(f"❌ 命令执行失败: {e.diag.message_primary}")
                    # 默认行为：遇到错误则抛出异常，触发 finally 中的回滚

            # 提交事务
            conn.commit()
            print("\n✅ 所有 SQL 命令在一个事务中成功执行并提交。")

        except psycopg2.OperationalError as e:
            print(f"\n❌ 连接或操作失败: {e}")
            print("请检查数据库配置、服务状态和网络连接。")
            
        except FileNotFoundError:
            print(f"\n❌ 错误: 找不到 SQL 文件 {SQL_FILE_PATH}，请检查路径。")

        except Exception as e:
            print(f"\n❌ 发生意外错误: {e}")
            if conn:
                conn.rollback()  # 遇到任何错误都回滚事务
                print("事务已回滚。")

        finally:
            # 关闭数据库连接
            if conn:
                cursor.close()
                conn.close()
                print("数据库连接已关闭。")


    execute_sql_file()

In [17]:
for i in subdir:
    commit2(i)

尝试连接到数据库: spider1_0_cre_doc_workflow...
连接成功。
读取 SQL 文件: __spider1_0_cre_doc_workflow-data.sql...
['INSERT INTO "authors" VALUES (\'Addison Denesik\',\'\'),(\'Adeline Wolff\',\'\'),(\'Antwon Krajcik V\',\'\'),(\'Beverly Bergnaum MD\',\'\'),(\'Bianka Cummings\',\'\'),(\'Dr. Dario Hermiston\',\'\'),(\'Dr. Shad Lowe\',\'\'),(\'Era Kerluke\',\'\'),(\'Eveline Bahringer\',\'\'),(\'Fiona Sipes DVM\',\'\'),(\'Jameson Konopelski\',\'\'),(\'Katharina Koepp\',\'\'),(\'Malvina Metz\',\'\'),(\'Marjolaine Paucek\',\'\'),(\'Mr. Joaquin Sanford\',\'\'),(\'Prof. Baron Heller II\',\'\'),(\'Shanie Skiles\',\'\'),(\'Telly Pfannerstill\',\'\'),(\'Tevin Weber\',\'\'),(\'Vidal Sanford\',\'\')', 'INSERT INTO "business_processes" VALUES (9,9,\'process\',\'normal\',NULL)', 'INSERT INTO "process_outcomes" VALUES (\'finish\',\'finish\'),(\'start\',\'starting soon\'),(\'working\',\'working on\')', 'INSERT INTO "ref_staff_roles" VALUES (\'ED\',\'Editor\'),(\'HR\',\'Human Resource\'),(\'MG\',\'Manager\'),(\'PR\',\'P